   <center><h1>This is a dev and test environment for EMD1.0</h1></center>

In [1]:
import os.path
'''Responsible for Writing Customized NetLogo models for EMD'''
class NetLogoWriter:
    __original_model_path = ""
    __rule_injected_model_path = ""
    __EMD_line = -1000
    
    '''locate the NetLogo file, read it in, and identify the line to be experimented with as EMD_line.'''
    def __init__(self, model_string):
        self.__original_model_path = model_string
        #find EMD entry point
        with  open(self.__original_model_path, 'r') as file_reader:#should probably catch an exception here

            for i, line in enumerate(file_reader):
                if ";EMD:EvolveNextLine;" in line:
                    print(i)
                    print(line)
                    self.__EMD_line = i + 1
                
    '''if EMD annotation exists, then inject the new rule string'''
    def injectNewRule (self, new_rule):
        # with is like your try .. finally block in this case
        with open(self.__original_model_path, 'r') as file:  #should probably catch an exception here
            # read a list of lines into data
            data = file.readlines()
            file.close()
        self.__rule_injected_model_path = self.__original_model_path[:-5] + new_rule + ".nlogo"
        if os.path.isfile(self.__rule_injected_model_path):
            print("Model already injected with this rule. Using cached version.")
        else:
            if self.__EMD_line >= 0:
                print( "Your line: " + data[self.__EMD_line])
                data[self.__EMD_line] = new_rule
                # and write everything back
                with open(self.__rule_injected_model_path, 'w') as file:
                    file.writelines( data )
                    file.flush()
                    file.close()
        return self.__rule_injected_model_path

In [2]:
import math 
import time
def EvaluateABM(modelname, setupCommands ,metricCommands, stopConditions, ticksToRun = -1):
    workspace = nl4py.netlogoWorkspaceFactory.newNetLogoHeadlessWorkspace()
    workspace.openModel(modelname)
    workspace.setParamsRandom()
    for setupCommand in setupCommands:     
        workspace.command(setupCommand)
    #workspace.command("setup")
    stopConditions.insert(0,"ticks")
    if ticksToRun < 0:
        ticksToRun = math.pow(2,31)
    workspace.scheduleReportersAndRun(stopConditions, 0,1,ticksToRun, "go")
    runDone = False
    metricResults = []
    while(runDone == False):
        time.sleep(0.5)
        #try:
        workspaceResults = workspace.getScheduledReporterResults()
        if len(workspaceResults) > 0:
            ticks = int(float(workspaceResults[-1][0]))
            stopConditionResults = workspaceResults[-1][1:]
            stop = False
            for stopConditionResult in stopConditionResults:
                if(str(stopConditionResult).lower() == "true"):
                    stop = True
                    break
            if (ticks == ticksToRun) or stop:
                for metricCommand in metricCommands:
                    metricResults.append(workspace.report(metricCommand))
                nl4py.netlogoWorkspaceFactory.getAllExistingWorkspaces().remove(workspace)
                workspace.deleteWorkspace()
                workspace = None
                runDone = True 
        #except Exception as inst:
            #print(type(inst))
            #print(inst.args)
            #print(inst)
    return metricResults

In [ ]:
import nl4py
nl4py.startServer()
bestError = 1000000000
bestRule = ""
rules = ["one-of sheep-here", "nobody", "min-one-of sheep [energy]", "max-one-of sheep [energy]"]
nl4py.netlogoWorkspaceFactory.deleteAllExistingWorkspaces()
for i in range(0,10):
    for rule in rules:
        modelPath = "./models/Wolf Sheep Predation.nlogo"
        modelWriter = NetLogoWriter(modelPath)
        modelPath = modelWriter.injectNewRule(rule)
        setup = ['set model-version "sheep-wolves-grass"', 'setup']
        measurementStrings = ["count sheep", "count wolves"]
        stopConditionStrings = ["not any? turtles","not any? wolves and count sheep > max-sheep"]
        result = EvaluateABM(modelPath, setup, measurementStrings, stopConditionStrings,100)
        aggregateMetric = abs(( int(float(result[0])) / (float(result[1]) + 0.0000001) ))
        if aggregateMetric < bestError:
            bestError = aggregateMetric
            bestRule = rule

In [104]:
from deap import gp
from deap import creator
from deap import base
from deap import tools

class nlMinOneOf:
    __nlString = ""
    
    def __init__(self,comparator, agent_set):
            self.__nlString = " min-one-of ( " + str(comparator) + " ) [ " +  str(agent_set) + " ] "
    def __str__(self):
        return self.__nlString
    def __repr__(self):
        return self.__str__()
class nlComparator:
    __nlString = ""
    def __init__(self,nlString):
        self.__nlString = nlString
    def __str__(self):
        return "nlComparator"#self.__nlString
    def __repr__(self):
        return self.__str__()
class nlAgentSet:
    __nlString = ""
    def __init__(self,nlString):
        self.__nlString = nlString
    def __str__(self):
        return "nlAgentSet"#self.__nlString
    def __repr__(self):
        return self.__str__()
class nlAgent:
    __nlString = "nobody"
    def __init__(self,nlString):
        self.__nlString = nlString
    def __str__(self):
        return "nlAgent" #self.__nlString
    def __repr__(self):
        return self.__str__()

pset = gp.PrimitiveSetTyped("main", [], nlAgent)
#pset.addPrimitive(operator.xor, [bool, bool], bool)
#pset.addPrimitive(operator.mul, [float, float], float)
#pset.addPrimitive(if_then_else, [bool, float, float], float)
pset.addPrimitive(nlMinOneOf, [nlComparator, nlAgentSet], nlAgent)
agent_sets = ["sheep","wolves","turtles"]
for agent_set in agent_sets:
    pset.addTerminal(agent_set, nlAgentSet, name = agent_set)
comparator_str_set = ["energy", "distance"]
for comparator_str in comparator_str_set:
    pset.addTerminal(comparator_str,nlComparator, name = comparator_str)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,pset=pset)
toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)


C:\Users\ch328575\AppData\Local\Continuum\Anaconda2\envs\Python3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\ch328575\AppData\Local\Continuum\Anaconda2\envs\Python3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [102]:
expr = gp.genGrow(pset, min_=1, max_=3, type_=nlAgent)
tree = gp.PrimitiveTree(expr)
print(str(tree))

nlMinOneOf(distance, wolves)


In [103]:
gp.compile(tree, pset)


TypeError: __repr__ returned non-string (type int)

In [ ]:
'''The ABM with all its flexibility as EMD sees it'''
'''EMD can use this class to control, set parameters, query results from'''
'''and most importantly, change rules within the NetLogo model specified'''
class ABM_NetLogo:
    path_to_model = ""
    model_as_text = ""
    parameters = []
    __nlogo_writer = None 
    def __init__(self, path_to_model):
        self.path_to_model = path_to_model
        __nlogo_writer = NetLogo_Writer(path_to_model)
        #Read in the model to memory (instead of repeated disk reads)
        print()
    def inject_new_rule(self, new_rule):
        #locate tag comment
        __nlogo_writer.injectNewRule(new_rule)
        #Replace line under tag with new_rule
        
        #Write model 
        
        #Flush, close
        print()
    def set_parameters(self, parameter_values):
        print()
    def get_parameter_names(self):
        #Return parameter names
        print()
    def run_abm(self):
        #Run the ABM
        print()


class EMD_Individual:
    def __init__(self):
        print()
    